## Sztuczna inteligencja i inżynieria wiedzy 
### Lista 2 - Implementacja MinMax dla gry Clobber
##### Aleksander Stepaniuk 272644

Raport dokumentuje strukturę projektu, kluczowe fragmenty kodu, napotkane problemy, użyte heurystyki i wnioski.

## 1. Struktura projektu
`clobber_app/`
- `main.py`: punkt wejścia, zbiera parametry wejściowe przez CLI lub GUI, uruchamia grę
- `game.py`: reprezentacja stanu gry, obsługa i generowanie możliwych ruchów
- `players.py`: definicje graczy (human, random, minimax)
- `heuristics.py`: funkcje heurystyczne
- `search.py`: minimax z alfa-beta
- `utils.py`: konwersja notacji, dekorator timing, wyświetlanie
- `logger_config.py`: kolorowany logger

## 2. Reprezentacja stanu gry (`game.py`)
Poniższa klasa `GameState` reprezentuje stan gry, w tym planszę i aktualnego gracza. Zawiera metody do inicjalizacji planszy oraz możliwych operacji na planszy, takich jak generowanie możliwych ruchów czy liczenie atakowanych pionków do wykorzystania w heurystykach.

In [ ]:
class GameState:
    def __init__(self, rows: int = 5, cols: int = 6):
        self.rows = rows
        self.cols = cols
        self.board = [[None]*cols for _ in range(rows)]
        self._init_board()
        self.current = 'B'  # 'B' = black (first), 'W' = white (second)

    def _init_board(self):
        for r in range(self.rows):
            for c in range(self.cols):
                self.board[r][c] = 'B' if (r+c) % 2 == 0 else 'W'

## 3. Heurystyki (`heuristics.py`)

Wyróżniłem cztery heurystyki:
- **mobility**: oblicza ilość atakowanych przez gracza pionków przeciwnika i jako heurystykę zwraca jeden przez tę liczbę dla reszty z dzielenia przez dwa równą jeden oraz minus jeden przez tę liczbę dla reszty z dzielenia przez dwa równą zero
- **opp_mobility**: 1 / (1 + liczba atakowanych naszych pionków przez oponenta)
- **centrality**: różnica sum odwrotności odległości od środka planszy gracza i przeciwnika
- **combined**: suma powyższych

Każda heurystyka obsługuje terminalne pozycje (czyli kiedy gra się kończy zwraca +∞ lub –∞).


In [ ]:
import math
INF = float('inf')

def _terminal_value(state, player: str):
    opp = 'W' if player == 'B' else 'B'
    if state.current == player and state.is_terminal():
        return -INF
    if state.current == opp and state.is_terminal():
        return INF

    return None

def mobility(state, player: str):
    term = _terminal_value(state, player)
    if term is not None:
        return term

    num = state.count_attacked_enemy_pieces(player)
    if num == 0:
        return -INF

    if num % 2 == 1:
        return 1.0 / num

    return -(1.0 / num)

def opponent_mobility(state, player: str):
    term = _terminal_value(state, player)
    if term is not None:
        return term
    opp = 'W' if player == 'B' else 'B'
    num = state.count_attacked_enemy_pieces(opp)
    # im mniej ruchów ma przeciwnik, tym większy wynik
    return 1.0 / (1 + num)

def centrality(state: GameState, player: str):
    term = _terminal_value(state, player)
    if term is not None:
        return term

    rows, cols = state.rows, state.cols
    center_r = (rows - 1) / 2
    center_c = (cols - 1) / 2
    max_dist = math.hypot(center_r, center_c)

    def score_for(p_char: str) -> float:
        total = 0.0
        for r in range(rows):
            for c in range(cols):
                if state.board[r][c] == p_char:
                    d = math.hypot(r - center_r, c - center_c)
                    total += (max_dist - d) / max_dist
        return total

    me = score_for(player)
    opp = score_for('W' if player == 'B' else 'B')
    return me - opp

def combined(state: GameState, player: str) -> float:
    term = _terminal_value(state, player)
    if term is not None:
        return term

    return mobility(state, player) + opponent_mobility(state, player) + centrality(state, player)

## 4. Minimax z Alfa-Beta (`search.py`)
- Implementacja rekurencyjna z funkcjami `max_value` i `min_value`
- Przechodzimy w pętli przez wszystkie możliwe ruchy, tworząc kopię stanu gry, następnie stosujemy ruch i wywołujemy na przemiennie `max_value` i `min_value` wewnątrz siebie dopóki nie osiągniemy głębokości lub stanu końcowego
- Funkcja zwraca najlepszy możliwy ruch oraz statystyki takie jak ilość odwiedzonych węzłów i czas wykonania

In [ ]:
import copy

class SearchResult:
    def __init__(self, move=None, value=None, stats=None):
        self.move = move
        self.value = value
        self.stats = stats or {'nodes': 0, 'time': 0}

def minimax_decision(state, depth, player, heuristic_fn, alpha_beta=False):
    stats = {'nodes': 0}

    def max_value(s, d, alpha, beta):
        stats['nodes'] += 1
        # implementacja max_value
        pass

    def min_value(s, d, alpha, beta):
        stats['nodes'] += 1
        # implementacja min_value
        pass

    best_val, best_move = -math.inf, None
    for m in state.get_legal_moves(player):
        s2 = copy.deepcopy(state)
        s2.apply_move(m)
        val = min_value(s2, depth-1, -math.inf, math.inf)
        if val > best_val:
            best_val, best_move = val, m

    return SearchResult(best_move, best_val, stats)

## 5. Napotkane problemy
- Pierwotne heurystyki `material` i `mobility` nie były adekwatne, ponieważ brakowało im sensu - `materiał` jest stały i nie zmienia się w trakcie gry, a `mobilność` licząca możliwą ilość dostępnych ruchów nie tworzyła podziału na ruchy gracza i przeciwnika, co prowadziło, że ich różnica była zawsze równa zero - dodatkowo `mobilność` nie miała do końca sensu jako heurystyka, bo wraz z biegiem gry ilość ruchów będzie się zmniejszać, w związku z tym heurystyka dążyłaby do przedłużania gry, a nie do wygrania
- Znalezienie sensownej heurystyki nie było łatwe, ponieważ gra Clobber jest specyficzna i nie ma wielu dostępnych materiałów, a jej zasady pomimo swej prostoty, utrudniają wymyślenie sensownej heurystyki
- Parsowanie notacji z formy szachowej do koordynatów w -> `notation_to_index` z wykorzystaniem regexa
- Implementacja minimax z alfa-beta była skomplikowana implementacyjnie przez konieczność śledzenia stanu gry i głębokości rekurencji

## 6. Wnioski
- Kluczowa jest dobra heurystyka do ograniczenia przestrzeni stanu
- Dobra heurystyka jest `kluczowa` do oceny pozycji na niedostatecznie dużej głębokości, ponieważ dla większych plansz nie jest możliwe przeszukiwanie całej przestrzeni stanów
- Obsługa terminalnych stanów (+∞/–∞) znacznie polepsza działanie algorytmu, ponieważ pozwala jednoznacznie stwierdzić, kiedy następuje `mat` czyli sytuacja, w której gracz nie może w żaden sposób uratować swojej sytuacji i musi wykonywać ruchy, aż do przegranej
- Podział na moduły pozwala na łatwe testowanie i rozwijanie kodu, a także łatwe dodawanie nowych strategii, heurystyk graczy oraz rozpoczynanie różnych starć np. gracz vs gracz albo gracz vs minimax albo minimax vs minimax albo minimax vs random itd. - każdy może grać z każdym